In [1]:
import numpy as np


In [2]:
# Function that counts the number of unique lables in a dataset
def countLabels(labels):
    lblCounter = {}
    for lbl in labels:
        if lbl not in lblCounter:
            lblCounter[lbl] = 0
        lblCounter[lbl] += 1
    return lblCounter


In [3]:
# Measures the GINI impurity in a dataset
# Read more at https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
def giniImpurity(features, labels):
    lblCount = countLabels(labels)
    impurity = 1
    for lbl in lblCount:
        probability = lblCount[lbl] / float(len(features))
        impurity -= probability**2
    return impurity


In [4]:
# Function to boot strap a dataset
def bootstrap(features, labels, sampleSize, numTrees):
    dataSize = len(features)
    bootstrapData = []
    for i in range(numTrees):
        # We want the sample indices to be chosen with replacement. randint, by default generates the sample with replacement
        bootstrapedIndices = np.random.randint(0, dataSize, sampleSize)
        bootstrapData.append({"features": features[bootstrapedIndices],
                    "labels": labels[bootstrapedIndices]})
    return bootstrapData




In [5]:
# Choose a feature randomly to slit the tree
def chooseFeature(featureNames, parentChoice):      
    currentChoice = np.random.choice(featureNames, replace=False)    
    while currentChoice == parentChoice:
        currentChoice = np.random.choice(featureNames, replace=False)
    
    return currentChoice




In [6]:
# Measure the information gain after splitting
# Read more at https://en.wikipedia.org/wiki/Decision_tree_learning#Information_gain
def informationGain(leftSize, rightSize, leftGINI, rightGINI, parentGINI):        
    return parentGINI - ( (leftSize/(leftSize + rightSize)) * leftGINI ) - ( (rightSize/(leftSize + rightSize)) * rightGINI )



In [7]:
# Function to create a single decision tree
def createDecisionTree(features, labels, parent=None, depth=0, side=None, tree=None):
    if tree is None:
        tree = {}

    # Choose a feature to split the tree for a given node
    currentChoice = chooseFeature(features.dtype.names, hash(parent))
    # The unique values for the selected feature
    uniqueMeasures = np.unique(features[currentChoice])

    # We need to identify the the value which will lead to the maximum information gain
    maxInformationGain = np.NINF
    bestMeasure = np.NINF    

    # Create two empty ndarray to store the subsets, if split on a measure / value
    leftNodeFeatures = np.empty_like(features)
    rightNodeFeatures = np.empty_like(features)

    leftNodeLabels = np.empty_like(labels)
    rightNodeLabels = np.empty_like(labels)

    # Iterate through each unique value to find which measure is best to split on
    for measure in uniqueMeasures:
        # A naive check. We place all values less than the current value to the left and other to the right
        condition = features[currentChoice] < measure
        leftNodeFeatures = features[(condition)]
        rightNodeFeatures = features[~(condition)]

        leftNodeLabels = labels[(condition)]
        rightNodeLabels = labels[~(condition)]

        # We calculate the gini impurities for the current dataset and if we split it based on the condition above
        parentGINI = giniImpurity(features, labels)
        leftGINI = giniImpurity(leftNodeFeatures, leftNodeLabels)
        rightGINI = giniImpurity(rightNodeFeatures, rightNodeLabels)    

        # We measure the information gain if we split the data set based on the condition
        gain = informationGain(len(leftNodeFeatures),
                            len(rightNodeFeatures), leftGINI, rightGINI, parentGINI)

        # If the gain is more than maximum information gain generated with other features and values, we update the max information gain and the best measure        
        if gain > maxInformationGain:
            maxInformationGain = gain
            bestMeasure = measure 

    # If there is information gain from splitting, we split the node
    if maxInformationGain > 0.0:    
        tree[hash(parent), depth, side] = {
            "isLeaf": False,
            "hash": hash(currentChoice),
            "feature": currentChoice,
            "measure": bestMeasure,
            "gain": maxInformationGain,
            "size": len(features)
        }

        # We  recursively split it into a binary tree
        createDecisionTree(
            features[(features[currentChoice] < bestMeasure)
                     ], labels[(features[currentChoice] < bestMeasure)],
                           currentChoice, depth+1, "left", tree)
        createDecisionTree(features[~(features[currentChoice] < bestMeasure)], labels[~(features[currentChoice] < bestMeasure)],
                           currentChoice, depth+1, "right", tree)
    else:
        # If there are no information gain from splitting, we append the leaf nodes   
        tree[hash(parent), depth, side] = {

            "isLeaf": True, 
            "label": np.unique(labels)[0]
            }
        
    return tree


In [8]:
# Function to generate multiple decision trees
def genForest(bootstrapedData):
    global forest
    forest = {}
    for i, data in enumerate(bootstrapedData):
        forest[i] = createDecisionTree(data['features'], data['labels'], None)
    
    return forest


In [9]:
# We fit the decision trees on the bootstrap data set
def fit(features, labels, sampleSize, numTrees=10, randomState=336):
    np.random.seed(randomState)
    bootstrapedData = bootstrap(features, labels, sampleSize, numTrees)    
    genForest(bootstrapedData)


In [10]:
# When evaluation the model each tree votes for a label for each record
def vote(record, tree, parent=None, depth=0, side=None, result=None):
    if result is None:
        result = []
    
    # Get the  node
    node = tree[hash(parent), depth, side]        
    if node['isLeaf']:        
        result.append(node['label'])
    else:
        if record[node['feature']] < node['measure']:
            vote(record, tree, node['feature'], depth+1, 'left', result)
        else:
            vote(record, tree, node['feature'], depth+1, 'right', result)
            
    return result[0]


In [11]:
# Collects the votes from all trees for a record and returns the label with the maximum vote
def predict(records, votes = None):
    if votes is None:
        votes = {}
    for i, record in enumerate(records):
        votes[i] = {}
        for index, tree in forest.items(): 
            result = vote(record, tree)
            if result not in votes[i]:
                votes[i][result] = 0            
            votes[i][result] += 1
    # Return the label with max vote. We have sorted the votes in decending order
    return [sorted(result.items(), key=lambda x: x[1], reverse=True)[0][0] for i, result in votes.items()]


In [12]:
# Evaluation our work, using the IRIS data set
import numpy 

# We create a custom data type for our iris dataset
cdtype = [("Id", int), ("SepalLengthCm", float), ("SepalWidthCm", float),
          ("PetalLengthCm", float), ("PetalWidthCm", float), ("Species", np.dtype('U25'))]


# Load the iris data set
with open("data/Iris.csv") as f:
    _data = f.readlines()

def func(record): return (int(record[0]), float(record[1]), float(
    record[2]), float(record[3]), float(record[4]), str(record[5]).replace('\n', ''))


data = np.array([func(l.split(',')) for l in _data[1:]], dtype=cdtype)

# Store the features and labels in two seperaate variables
features = data[["SepalLengthCm", "SepalWidthCm",
                 "PetalLengthCm", "PetalWidthCm"]]
labels = data["Species"]


In [13]:
# We need to split the dataset into train and test sets. We select the indices that would be included in each set
trainIndices = np.random.choice(range(len(features)), size=int(0.8*len(features)), replace=False)
testIndices = list(set(range(len(features))).difference(trainIndices))


In [14]:
# Prepare the training set
trainFeatures = features[trainIndices]
trainLabels = labels[trainIndices]
print("length of train features:", len(trainFeatures), "and length of train labels:", len(trainLabels))

# Train the forest on the training set
fit(trainFeatures, trainLabels, sampleSize=50, numTrees=10)


length of train features: 120 and length of train labels: 120


In [15]:
# predict on the training set. We can understand how strong is our model
trainPredict = predict(trainFeatures)
print("length of train predict:", len(trainPredict))


length of train predict: 120


In [16]:
# Training set score. Evaluating the model
score = np.sum(trainLabels == trainPredict)
print("Score:", score/len(trainLabels) * 100)

Score: 99.16666666666667


In [17]:
# Testing on unseen data. First we preparing the test set
testFeatures = features[testIndices]
testLabels = labels[testIndices]
print("length of test features:", len(testFeatures),
      "and length of test labels:", len(testLabels))


length of test features: 30 and length of test labels: 30


In [18]:
# predict on the testing set
testPredict = predict(testFeatures)
print("length of train predict:", len(testPredict))


length of train predict: 30


In [19]:
#Testing the score
score = np.sum(testLabels == testPredict)
print("Score:", score/len(testLabels) * 100)


Score: 86.66666666666667


In [20]:
#Let us display the test dataset with the actual and predicted values as a pandas dataset
import pandas as pd

In [21]:
testOut = pd.concat((pd.DataFrame(testFeatures), pd.DataFrame(testLabels), pd.DataFrame(testPredict)), axis=1)
testOut.columns = ["Sepal Length (cm)", "Sepal Width (cm)", "Patel Length (cm)",
                 "Patel Width (cm)", "Species (actual)", "Species (predicted)"]
testOut


,Sepal Length (cm),Sepal Width (cm),Patel Length (cm),Patel Width (cm),Species (actual),Species (predicted)
0,4.7,3.2,1.3,0.2,Iris-setosa,Iris-setosa
1,5.0,3.6,1.4,0.2,Iris-setosa,Iris-setosa
2,5.4,3.9,1.7,0.4,Iris-setosa,Iris-setosa
3,4.4,2.9,1.4,0.2,Iris-setosa,Iris-setosa
4,5.7,4.4,1.5,0.4,Iris-setosa,Iris-setosa
5,5.7,3.8,1.7,0.3,Iris-setosa,Iris-setosa
6,5.1,3.8,1.5,0.3,Iris-setosa,Iris-setosa
7,4.6,3.6,1.0,0.2,Iris-setosa,Iris-setosa
8,5.1,3.3,1.7,0.5,Iris-setosa,Iris-setosa
9,5.2,3.5,1.5,0.2,Iris-setosa,Iris-setosa
